# Food and Agricultural Trends

**Team Papillon**

Nowadays, environmental problems are becoming more and more serious and urgent to deal with. We cannot deny the impact of food and agriculture in general, on the environment. If we consider what we eat and how we grow it, we would find extensive damage to the environment (green gas emissions, soil depletion etc.) and also the wildlife (due to pesticides, fertilizers etc.). We are looking to use the data provided by FAOSTAT giving access to over 3 million time-series and cross sectional data relating to food and agriculture all over the world and try to generate insights and stories on the evolution of different socio-environmental factors such as the correlation between greenhouse gas emissions and agricultural growth for example. Through this work, we hope to gain a deeper insight on the evolution and the environmental impact of agriculture and food.

## Question 1 and 2

### Could we find and support a correlation between greenhouse gas emissions and agricultural growth? Could we also assess the influence of the type of culture on the emissions?
### Based on this, is it possible to make predictions of greenhouse gas emissions by extrapolating the agricultural growth and land usage?

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import statsmodels
import folium
import math

# Custom imports
from ipywidgets import IntProgress
from IPython.display import display
import time
from multiprocessing import Pool, Lock
import os
import json
import seaborn as sns
import time
from scipy import stats

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing   import StandardScaler

## Question 1 and 2

### Could we find and support a correlation between greenhouse gas emissions and agricultural growth? Could we also assess the influence of the type of culture on the emissions?
### Based on this, is it possible to make predictions of greenhouse gas emissions by extrapolating the agricultural growth and land usage?

## Importing the data

In order to find a model that helps in the understanding of the role of the variety of crops in the ecological influence of the agriculture, we need 3 datasets:

- The land usage
- The crops cultures
- The emissions related to the agriculture


In [ ]:
dataLands = pd.read_csv("./data/fao_data_land_data.csv")

dataCrops = pd.read_csv("./data/fao_data_crops_data.csv")

dataEmissions = pd.read_csv("./data/current_FAO/raw_files/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv", encoding="cp1252")

## Cleaning

- Removing NAN
- Removing useless columns

In [ ]:
dataLands = dataLands.dropna(subset=["element"])

dataCrops = dataCrops.dropna(subset=["element"])

dataEmissionsAgriculture = dataEmissions.where(dataEmissions["Item"] == "Agriculture total").where(dataEmissions["Element"] == "Emissions (CO2eq)").dropna()
dataEmissionsAgriculture = dataEmissionsAgriculture.drop(["Item", "Element Code", "Element", "Item Code", "Year Code", "Flag"], axis=1)\
                                                    .rename(columns={"Unit":"Unit emissions","Value":"Value emissions"})

Here is a graph that shows the progression of land usage within each continent.

In [ ]:
def cond_countries(dataLands):
    countries = ["Asia +","Europe +", "Americas +", "Oceania +", "Africa +"]
    truthTable = (dataLands["country_or_area"] == countries[0])
    for c in countries:
        truthTable = (dataLands["country_or_area"] == c) | truthTable
    return truthTable

dataLandsContinent = dataLands.where(dataLands["category"] == "agricultural_area")\
                                .where(cond_countries(dataLands))\
                                .dropna(subset=["country_or_area"])\
                                .sort_values("value",ascending=False)

sns.set(style="darkgrid")

fg = plt.figure(figsize=(20,20))
axes = fg.add_subplot()
# Plot the responses for different events and regions
sns.lineplot(x="year", y="value", style="country_or_area", data=dataLandsContinent, hue="country_or_area", ax=axes)

fg.show()

Here are the emissions of the BRICS throughout the years. We can see a clear progression (except for South Africa).

In [ ]:
sns.set(style="darkgrid")

dataEmissionsAgrContinent = dataEmissionsAgriculture.where(dataEmissionsAgriculture["Area"].isin(["South Africa","Brazil", "China", "India", "USSR","Russian Federation"])).dropna()

fg = plt.figure(figsize=(20,20))
axes = fg.add_subplot()
# Plot the responses for different events and regions
sns.lineplot(x="Year", y="Value emissions", style="Area", data=dataEmissionsAgrContinent, hue="Area", ax=axes)

fg.show()

## Creating the dataset for processing

The dataset we create for processing consist in the aggregation of the area harvested of for each crop, by year and by country, and the emissions of greenhouse gases by year and by country as well. The goal will be to find a model that explain and helps in predicting the emissions, based on the other features (country, year and crops).

In [ ]:
crops = dataCrops.drop(["element_code"], axis=1)\
                .where(dataCrops["element"] == "Area Harvested")\
                .dropna(subset=["element"])\
                .drop(["element", "value_footnotes"],axis=1)\
                .rename(columns={"unit":"Unit area", "value":"Value area", "year":"Year","country_or_area":"Area"})
#                                .where(dataCrops["country_or_area"] == "World +")\
cropsAndEmissions = crops.pivot_table(values='Value area',index=["Area","Year"],columns="category").reset_index()
cropsAndEmissions = cropsAndEmissions.fillna(0)
cropsAndEmissions = pd.merge(cropsAndEmissions, dataEmissionsAgriculture, how="left", on=['Area',"Year"])#.dropna(subset=["Value area", "Value emissions"])
cropsAndEmissions = cropsAndEmissions.dropna().dropna(subset=["Unit emissions"])
cropsAndEmissions

## Learning to predict emissions

Here, we try to create a model by doing a logistic regression on the data. We first try to optimize the alpha parameter on a predefined interval and then we use it to train the model.

In [ ]:
SEED = 1
st_pipeline = Pipeline([('scl', StandardScaler()), ('ridge', Ridge(copy_X=True, random_state=SEED))])
st_pipeline

#features = ["Area","category"]

XRidge2 = pd.get_dummies(cropsAndEmissions.drop(["Area Code","Unit emissions", "Value emissions"], axis=1))
yRidge2 = cropsAndEmissions["Value emissions"]

results = []

# Tune for alpha using 10 fold crossvalidation when calculating the mean squared error.
for alpha in np.linspace(-100, 100, 32):
    st_pipeline.set_params(ridge__alpha= alpha) 
    neg_MSE = cross_val_score(st_pipeline, XRidge2, yRidge2, scoring='neg_mean_squared_error', cv=10)  # we use 10 folds crossvalidation since 10 
                                                                                                      # is pretty much standard in the industry
    results.append([neg_MSE, alpha])
    
# Take the mean MSE for each level of alpha
for i in range(len(results)):
    results[i][0] = -np.mean(results[i][0])
    
# Plot the results
plt.figure(figsize=(20,20))
plt.plot([row[1] for row in results], [row[0] for row in results])
plt.xlabel('alpha')
plt.ylabel('MSE')
plt.title('Tuning alpha');
plt.grid()

best_st_alpha = min(results)[1]
print('Best alpha is', best_st_alpha, 'with a MSE of', min(results)[0],'.')

In [ ]:
st_Model = st_pipeline.set_params(ridge__alpha= best_st_alpha)  # use best alpha calculated above
st_Model.fit(XRidge2, yRidge2)                                  # fit the new model

In [ ]:
#st_Model.named_steps["ridge"].coef_
weights = pd.DataFrame([XRidge2.columns,st_Model.named_steps["ridge"].coef_]).transpose().sort_values(1)
weights

Further analysis will be conducted in order to create efficient predictions. This first analysis already gave us clear insight of the data and helps us in assessing the feasability of this research.

## Plan

The next steps will be :

- Visualize the output of the model and its relation to reality.
- Try new models and assess their performances.
- Make predictions on the future using these regressions.
- Extract guidelines that could help in reducing emissions in the future, based on these data.
- Criticize these guidelines and the predictions using more domain-related knowledge.

# Output of the model

In [ ]:
weights.columns = ["name", "weight"]

In [ ]:
fg = plt.figure(figsize=(40,20))

axes1 = fg.add_subplot(3,1,1)
sns.barplot(x="name", y="weight", data=weights, ax=axes1)
axes1.set_title("All weights")

axes2 = fg.add_subplot(3,1,2)
sns.barplot(x="name", y="weight", data=weights.where(weights.name.str.contains("Area")).dropna(), ax=axes2)
axes2.set_title("Weights of the countries")

axes3 = fg.add_subplot(3,1,3)
sns.barplot(x="name", y="weight", data=weights.mask(weights.name.str.contains("Area")).mask(weights.name == "Year").dropna(), ax=axes3)
axes3.set_title("Weights of the crops")

fg.tight_layout()

In [ ]:
weights.where(weights.name.str.contains("Area")).where(weights.weight > 0).dropna().sort_values("weight")#.where("weight > 0")

In [ ]:
weights.mask(weights.name.str.contains("Area")).where(weights.weight > 0).dropna().sort_values("weight").head()

## New model for predictions

In order to obtain better predictions, we decided to restart the process. Using linear regressions, we compute the emissions produced by each crops.

In [ ]:
crops = dataCrops.drop(["element_code"], axis=1)\
                .where(dataCrops["element"] == "Area Harvested")\
                .mask(dataCrops.country_or_area.isin(["Asia +","Europe +", "Americas +", "Oceania +", "Africa +"]))\
                .dropna(subset=["element"])\
                .drop(["element", "value_footnotes"],axis=1)\
                .rename(columns={"unit":"Unit area", "value":"Value area", "year":"Year","country_or_area":"Area"})
crops.head(5)

In [ ]:
listCountries = ["United States of America", "India", "Switzerland", "United Kingdom", "China"]
filterCrops = ["cereals_total", "cereals_rice_milled_eqv","pulses_total", "coarse_grain_total", "treenuts_total"]

#cropsEmissionsNormalized = crops.where(crops.Area.isin(listCountries)).dropna()
cropsEmissionsNormalized = crops
totalHarvestedArea = cropsEmissionsNormalized.groupby(["Area","Year"]).sum()#.drop(["Year"], axis=1)#.rename("Value area", "Total area harvested")
totalHarvestedArea.columns = ["Total area harvested"]
cropsEmissionsNormalized = pd.merge(cropsEmissionsNormalized, totalHarvestedArea, how="left", on=['Area',"Year"])
cropsEmissionsNormalized = pd.merge(cropsEmissionsNormalized, dataEmissionsAgriculture, how="left", on=['Area',"Year"])#.dropna(subset=["Value area", "Value emissions"])
cropsEmissionsNormalized = cropsEmissionsNormalized.dropna().dropna(subset=["Unit emissions"])
cropsEmissionsNormalized["Value emissions"] = cropsEmissionsNormalized["Value area"]*cropsEmissionsNormalized["Value emissions"]/(cropsEmissionsNormalized["Total area harvested"]*cropsEmissionsNormalized["Total area harvested"])
cropsEmissionsNormalized = cropsEmissionsNormalized.mask(cropsEmissionsNormalized.category.isin(filterCrops)).dropna()
cropsEmissionsNormalized.head(5)

**cropsEmissionsNormalized** is a dataframe containing the value of emissions per crops, per country and per year. The total emissions for a given country and a given year was split over the different crops using their ratio of harvested area over the total harvested area. Then, these emissions were again normalized by the total harvested area, in order to obtain comparable results above all the countries. 

In [ ]:
fg = plt.figure(figsize=(15,12))

i = 1

listCrops = ["wheat","soybeans", "apples", "peas_dry"]

for c in listCrops:
    #print(cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).dropna())
    axes = fg.add_subplot(len(listCrops),1,i)
    sns.lineplot(x="Year", y="Value area",\
                data=cropsEmissionsNormalized.where(cropsEmissionsNormalized.category == c).dropna(),\
                ax=axes)
    axes.set_title(c)
    i = i+1

fg.tight_layout()

On the graph above, you can see the evolution in time of the *area harvested* for different crops. Except for *wheat*, there is a slight increase for each of the above. It is strange if we compare this to the evolution of the population, which more or less doubled during the last 3 decades. This could be explained by the industrialization of the agriculture, which requires less area to produce a similar amount of food.

In [ ]:
fg = plt.figure(figsize=(15,12))

i = 1

listCrops = ["wheat","soybeans", "apples", "peas_dry"]

for c in listCrops:
    #print(cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).dropna())
    axes = fg.add_subplot(len(listCrops),1,i)
    sns.lineplot(x="Year", y="Value area", hue="Area",\
                data=cropsEmissionsNormalized.where(cropsEmissionsNormalized.category == c)\
                 .where(cropsEmissionsNormalized.Area.isin(listCountries)).dropna(),\
                ax=axes)
    axes.set_title(c)
    i = i+1

fg.tight_layout()

This graph gives a preview of the evolution of these crops for each country. They all follow the overall evolution and none of them demonstrate a clear increase or decrease in the harvesting of these crops.

In [ ]:
fg = plt.figure(figsize=(30,20))

i = 1

for c in listCountries:
    #print(cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).dropna())
    axes = fg.add_subplot(5,1,i)
    try:
        sns.barplot(x="category", y="Value emissions",\
                    data=cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).where(cropsEmissionsNormalized["Value emissions"] > 0.00005).dropna(),\
                    ax=axes)
    except ValueError as e:
        print(str(e))
    axes.set_title(c)
    i = i+1

fg.tight_layout()

In the plot above, we show the **normalized emissions values** for the considered countries and we only display the crops that are the most "emitting". One very interesting fact is that Switzerland as well as UK seems to have a really considerable number of crops they grow that emits over the threshold we set. In order to investigate this, we made the graph below.

In [ ]:
fg = plt.figure(figsize=(15,15))

i = 1

for c in listCrops:
    #print(cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).dropna())
    axes = fg.add_subplot(len(listCrops),1,i)
    try:
        sns.barplot(x="Area", y="Value emissions",\
                    data=cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area.isin(listCountries)).where(cropsEmissionsNormalized["category"] == c).dropna(),\
                    ax=axes)
    except ValueError as e:
        print(str(e))
    axes.set_title(c)
    axes.set_ylabel("Emissions [Gg/ha]")
    i = i+1

fg.tight_layout()

fg.savefig("emissions_by_unit_area_crop.eps")

This graph shows for 4 different crops, their normalized emissions value, aggregated in time. This is very insightful as it appears systematically that producing one of this type of crop in Switzerland, in UK or in US for the soybeans, emits more greenhouse gas than producing this in China or in India. This could be the consequence of the size of the production. Lets now compare the area harvested for each of these crops in these countries.

In [ ]:
fg = plt.figure(figsize=(15,15))

i = 1
listCrops = ["wheat","soybeans", "apples", "peas_dry"]

for c in listCrops:
    #print(cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).dropna())
    axes = fg.add_subplot(len(listCrops),1,i)
    try:
        sns.barplot(x="Area", y="Value area",\
                    data=cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area.isin(listCountries)).where(cropsEmissionsNormalized["category"] == c).dropna(),\
                    ax=axes)
    except ValueError as e:
        print(str(e))
    axes.set_title(c)
    axes.set_ylabel("Area [ha]")
    i = i+1

fg.tight_layout()
fg.savefig("area_crop.eps")

Here you clearly see that the production in China and in India are considerably bigger than those in United Kingdom or Switzerland. Therefore, we could argue that producing more is more efficient. However, the case of soybeans counterbalance this argument as the US is at the same time the biggest producer and least efficient producer of soybeans in terms of gas emissions. This could be put in relation with many other factors such as:

- human vs machine use in fields,
- automatisation,
- technical knowledge in agriculture,
- fertilizer and pesticide use.

## Create a model to assess the environmental efficiency for each crop

We will know create a model to predict the efficiency of each crop. This model will then be used to compute the emissions related to each crop in the near future. Based on this, we will also create a model to predict the agriculture type in each country in terms of crop development and area use in the next years. By combinig these two models, we will then try to predict the overall emissions for each country.

In [ ]:
def create_model_for_crop(crop):
    cropData = cropsEmissionsNormalized.where(cropsEmissionsNormalized.category ==crop).dropna()
    lm = LinearRegression()
    
    cropData = cropData.groupby("Year").sum().reset_index()

    X = cropData[["Year"]]
    Y = cropData[["Value emissions"]]

    lm.fit(X,Y)
    
    return lm

models = []

for crop in cropsEmissionsNormalized.category.unique():
    models.append((crop,create_model_for_crop(crop)))
    
cropsModelsEmissionsPerUnitArea = pd.DataFrame(models, columns=("crop","model_emissions"))

In [ ]:
fg = plt.figure(figsize=(10,10))

i = 1
listCrops = ["wheat","soybeans", "apples", "peas_dry"]

for c in listCrops:
    #print(cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).dropna())
    axes = fg.add_subplot(2,2,i)
    cropData = cropsEmissionsNormalized.where(cropsEmissionsNormalized["category"] == c)\
                                        .dropna()\
                                        .groupby("Year").sum().reset_index()
    try:
        sns.regplot(x="Year", y="Value emissions",\
                    data=cropData,\
                    ax=axes)
    except ValueError as e:
        print(str(e))
    axes.set_title(c)
    i = i+1

fg.tight_layout()
fg.savefig("emissions_per_crop_prediction.pdf")

As it can be seen, some crops have a strong linear dependency while other crops can have very sparse data and a probably difficult to predict behaviour. In some cases it is very relevant to make linear regressions. For the sake of simplicity, we will keep the linear regressions for all the crops.

In [ ]:
def get_model_crop(crop, cropsData):
    cropsData = cropsData.where(cropsData.category ==crop).dropna()
    
    lm = LinearRegression()
    
    cropsData = cropsData.groupby("Year").sum().reset_index()
    
    X = cropsData[["Year"]]
    Y = cropsData[["Value area"]]
    
    try:
        lm.fit(X,Y)
    except ValueError as e:
        #print(str(e))
        return np.nan
    
    return lm

def get_model_crops_country(country):
    cropsData = cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area ==country).dropna()
    
    models = []

    for crop in cropsEmissionsNormalized.category.unique():
        models.append((crop,get_model_crop(crop, cropsData)))
    
    cropsModelsGrowth = pd.DataFrame(models, columns=("crop","model_growth"))
    
    return cropsModelsGrowth

class PredictorEmissionsCountry():
    def __init__(self, country, modelCrops, modelEmissionsByCrops):
        self.country = country
        self.modelCrops = modelCrops
        self.modelEmissionsByCrops = modelEmissionsByCrops
        
    def predict_emissions(self, startYear, endYear):
        years = []
        for i in range(startYear, endYear):
            years.append(i)
            
        def compute_production_area(row):
            return (row.model_growth.predict(pd.DataFrame(years)),)
        def compute_emissions_per_unit_area(row):
            return (row.model_emissions.predict(pd.DataFrame(years)),)
        def compute_emissions_per_unit_area(row):
            return (row.model_emissions.predict(pd.DataFrame(years)),)
        def compute_emissions(row):
            return (row.production_area*row.emissions_per_unit_area,)
        def flatten(arr):
            flatArr = []
            for i in arr:
                flatArr.append(i[0])
            return flatArr

        m = pd.merge(self.modelCrops, self.modelEmissionsByCrops, how="left", on=['crop']).dropna()
        m = m.join(m.apply(compute_production_area, axis=1, result_type="expand")).rename(columns={0:"production_area"})
        
        m = m.join(m.apply(compute_emissions_per_unit_area, axis=1, result_type="expand")).rename(columns={0:"emissions_per_unit_area"})
        m = m.join(m.apply(compute_emissions, axis=1, result_type="expand")).rename(columns={0:"emissions"})
        m.emissions = m.emissions.apply(flatten)
        m = pd.DataFrame(m.emissions.tolist(), columns=years)

        m = m.sum(0)
        return pd.DataFrame(m,columns=["emissions"]).reset_index().rename(columns={"index":"year"})

def get_model_emissions_countries(listCountries):
    #get_model_crops_country(country)
    
    predictorsEmissions = []

    for country in cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area.isin(listCountries)).dropna().Area.unique():
        predictorsEmissions.append(PredictorEmissionsCountry(country,\
                                                            get_model_crops_country(country),\
                                                            cropsModelsEmissionsPerUnitArea))
        print("Predictor for " + country + " done.")
    
    return predictorsEmissions

In [ ]:
models = get_model_emissions_countries(listCountries)

fg = plt.figure(figsize=(15,15))

i = 1

for mc in models:
    #print(cropsEmissionsNormalized.where(cropsEmissionsNormalized.Area == c).dropna())
    emissions = mc.predict_emissions(2000,2040)
    axes = fg.add_subplot(5,1,i)
    try:
        sns.lineplot(x="year", y="emissions",data=emissions,\
                    ax=axes)
    except ValueError as e:
        print(str(e))
    axes.set_title(mc.country)
    axes.set_ylabel("Emissions [Gg]")
    i = i+1

fg.tight_layout()
fg.savefig("emissions_prediction.eps")

Using the model above, these are the predictions for the emissions of the selected countries. It is very impressive to see how China and India, which could still be considered as developping countries if we consider the life quality of the poorest part of the population, have continuously increasing emissions. However, more developped countries such as US, UK and Switzerland seems to be able to mitigate and reverse the growth of their emissions.

# **Question 3 initial exploration**

Is it possible to rank some patterns of land usage with social and environmental factors such as employment, life satisfaction (if data are presents), income inequalities (if data are presents), emissions and finally soil quality and sustainability?

To answer this question we will need to acquire additionnal data, but before doing that, we must define the **scope** of the question. It doesn't really make sense to aggregate countries into continents since the social factors are really specific to each country. Therefore, we reduce our scope to a list of 12 countries that we are interested in and feel are important.

In [ ]:
countries = ['France', 'Canada', 'Germany', 'India', 'Japan', 'Russian Federation', 'Switzerland', 'United States of America', 'United Kingdom', 'China', 'Israel']

## Data acquisition and cleaning

In [ ]:
dataLands = pd.read_csv("./data/fao_data_land_data.csv")
dataLands.head()

In [ ]:
# investigate NaN values
dataLands[np.isnan(dataLands['value'].values)].head(3)

We observe that the dataset is pretty clean. The only NaNs in 'value' column are actually the footnotes so we can drop them.

In [ ]:
dataLands = dataLands.dropna(subset=["value"])
dataLands.tail(3)

In [ ]:
print(dataLands['category'].unique().tolist())

# we only keep agricultural area
dataLands = dataLands[dataLands['category'] == 'agricultural_area']

In [ ]:
# let's observe the temporal evolution of agricultural lands for our selected countries

def check_country(countries, DF):
    """Checks wether the countries 'countries' are in the dataframe.series DF"""
    for country in countries:
        if country not in DF.unique():
            print('Country', country, 'is missing!')
        
check_country(countries, dataLands['country_or_area'])

# reduce subset to selected countries
rdataLands = dataLands.loc[dataLands['country_or_area'].isin(countries)]

# groupe by country
grdataLands = rdataLands.groupby('country_or_area')

plt.figure(figsize=(15,15))
for country in countries:
    # we normalize the total land area by devinding by its mean
    sns.lineplot(x=grdataLands.get_group(country).year, y=grdataLands.get_group(country).value/np.mean(grdataLands.get_group(country).value), label=country)
    
plt.ylabel('land area / mean land area')
plt.title('Relative evolution of agricultural land areas');

We observe an overall downward trend the agricultutral land areas except for a few countries (India and China). However, we also notice an anomally with Russia that must be investigated: no data is present before 1991.

After some research we found out that the dissolution of the Soviet Union (USSR) took place on 26 December 1991, creating the country Russia. We must correct our data:

In [ ]:
display(dataLands[dataLands['country_or_area'] == 'USSR'].head(3))

# we don't really care about USSR, therefore, we rename it to Russia and subtract (USSR_land_1991 - Russia_land_1992) from its values
to_subtract = dataLands[(dataLands['country_or_area'] == 'USSR') & (dataLands['year'] == 1991)].value.values - dataLands[(dataLands['country_or_area'] == 'Russian Federation') & (dataLands['year'] == 1992)].value.values
# append rdataLands
for year,value in zip(dataLands[dataLands['country_or_area'] == 'USSR'].year.values, dataLands[dataLands['country_or_area'] == 'USSR'].value.values):
    rdataLands = rdataLands.append({'country_or_area':'Russian Federation', 'year':year, 'value':value - to_subtract[0]}, ignore_index=True)

In [ ]:
# check the graph again
# groupe by country
grdataLands = rdataLands.groupby('country_or_area')

plt.figure(figsize=(15,15))
for country in countries:
    # we normalize the total land area by devinding by its mean
    sns.lineplot(x=grdataLands.get_group(country).year, y=grdataLands.get_group(country).value/np.mean(grdataLands.get_group(country).value), label=country)
    
plt.ylabel('land area / mean land area')
plt.title('Relative evolution of agricultural land area')

In [ ]:
plt.figure(figsize=(20,5))
for country in countries:
    plt.bar(x=country, height=np.mean(grdataLands.get_group(country).value), label = country)
plt.legend()
plt.title('Mean agriculture land area');

In [ ]:
rdataLands.to_pickle('dataLand.pkl')

In [ ]:
# Soil erosion data
dataSoil = pd.read_csv('data/current_FAO/raw_files/Environment_Soil_E_All_Data.csv', encoding = "ISO-8859-1")
#display(dataSoil.head())

# check for our countries
print(check_country(countries, dataSoil['Country']))

# reduce to selected countries
rdataSoil = dataSoil.loc[dataSoil['Country'].isin(countries)]
rdataSoil.head()

Except the carbon content all the other data (erosion and lan degredation) haave only 1 measure in 1991 for each country.

In [ ]:
dataSoil.to_pickle('dataSoil.pkl')

## **New Data**
In order to answer the question at the beginning we need to search for and acquire some new data:

### Life expectancy at birth for both sexes combined (years):
http://data.un.org/Data.aspx?d=PopDiv&f=variableID%3a68

In [ ]:
lifeExp_df = pd.read_csv('data/life_expectancy.csv')
lifeExp_df.head(3)

In [ ]:
# rename columns for consistency
lifeExp_df.rename(columns={'Country or Area': 'country_or_area', 'Year(s)': 'years', 'Variant': 'variant', 'Value': 'value'}, inplace= True)

# Check our list
print(check_country(countries, lifeExp_df['country_or_area']))

# reduce to our selected countries
rlifeExp_df = lifeExp_df.loc[lifeExp_df['country_or_area'].isin(countries)]

# do we have the same problem as before with russia ?
rlifeExp_df[rlifeExp_df['country_or_area']=='Russian Federation'].tail()

Fortunately we don't have the above problem with Russia in this dataset. We observe that 'Years' is a contains 5 years long periods, thus we transform it and erase the second year (-19**). 

In [ ]:
# clean column year
rlifeExp_df['year'] = rlifeExp_df['years'].str.split('-', expand=True).iloc[:,0].astype('int')
rlifeExp_df.drop(columns='years', inplace= True)
rlifeExp_df = rlifeExp_df[rlifeExp_df['year'] < 2019] # we are not interested in predictions

In [ ]:
# simple observations
# groupe by country
grlifeExp_df = rlifeExp_df.groupby('country_or_area')

plt.figure(figsize=(15,15))
for country in countries:
    sns.lineplot(x=grlifeExp_df.get_group(country).year, y=grlifeExp_df.get_group(country).value, label=country)
    
plt.ylabel('Life expectancy')
plt.title('Evolution of life expectancy');

We see that india and china had a big evolution from the 1960s. However, China is the only country that also had a similar evolution in its agricultural lands.

In [ ]:
rlifeExp_df.to_pickle('lifeExp.pkl')

### Value added by industries at current prices (ISIC Rev. 3)

We were unable to download all the data from the below website, therefore, we reduced the industries to agriculture.

http://data.un.org/Data.aspx?d=SNA&f=group_code%3a201

In [ ]:
valueAdded_df = pd.read_csv('data/value_added.csv', low_memory=False)
valueAdded_df = valueAdded_df.dropna(subset=['Value'])
valueAdded_df.head(3)

In [ ]:
# rename columns for consistency
valueAdded_df.rename(columns={'Country or Area': 'country_or_area', 'Year': 'year', 'Value': 'value'}, inplace= True)

# Check our list
print(check_country(countries, valueAdded_df['country_or_area']))

In [ ]:
# print all countries
print(valueAdded_df['country_or_area'].unique().tolist())

In [ ]:
# Correct for united states
valueAdded_df.replace(to_replace='United States', value='United States of America', inplace= True)

# reduce to our selected countries
rvalueAdded_df = valueAdded_df.loc[valueAdded_df['country_or_area'].isin(countries)]

# do we have the same problem as before with russia ?
rvalueAdded_df[rvalueAdded_df['country_or_area']=='Russian Federation'].tail()

In [ ]:
# simple observations
# groupe by country
gvalueAdded_df = valueAdded_df.groupby('country_or_area')

plt.figure(figsize=(15,15))
for country in countries:
    # plot  and normalize by mean
    sns.lineplot(x=gvalueAdded_df.get_group(country).year, y=gvalueAdded_df.get_group(country).value/np.mean(gvalueAdded_df.get_group(country).value), label=country)
    
plt.ylabel('added-value')
plt.title('Economic added value by agriculture');

We see a strange peek with Russia that will have to be investigated for the final milestone. Russia's data starts from 1990 so we might have the same problem as before, but USSR is absent from the data...

In [ ]:
valueAdded_df.to_pickle('valueAdded.pkl')

### Employment by sex and economic activity

https://www.ilo.org/shinyapps/bulkexplorer5/?lang=en&segment=indicator&id=EMP_TEMP_SEX_ECO_NB_A

In [ ]:
employment_df = pd.read_csv('data/Employment.csv')
display(employment_df.head(3))
employment_df = employment_df.drop(columns=['obs_status.label', 'note_classif.label', 'note_indicator.label'])   # drop useless columns

In [ ]:
# rename columns for consistency
employment_df.rename(columns={'ref_area.label': 'country_or_area', 'time': 'year', 'obs_value': 'value', 'classif1.label': 'activity'}, inplace= True)

# Check our list
print(check_country(countries, employment_df['country_or_area']))

# Correct for united states
employment_df.replace(to_replace='United States', value='United States of America', inplace= True)

# reduce to our selected countries
remployment_df = employment_df.loc[employment_df['country_or_area'].isin(countries)]

# USSR isn't present: no problem
np.sort(employment_df['country_or_area'].unique())[-20:]

In [ ]:
# do a sub selection of only agricultural related economic activites
remployment_df = remployment_df[remployment_df['activity'].str.contains('Agriculture')]

In [ ]:
# extra cleaning
remployment_df.drop(columns=['indicator.label', 'source.label', 'note_source.label'], inplace= True)   # drop useless columns
remployment_df.replace({'Sex: Male': 'male', 'Sex: Female': 'female', 'Sex: Total': 'total'}, inplace= True)

In [ ]:
remployment_df.head()

In [ ]:
# simple observations
# groupe by country
gremployment_df = remployment_df[remployment_df['sex.label'] == 'total'].groupby('country_or_area')

plt.figure(figsize=(15,15))
for country in countries:
    # plot and normalize by mean
    sns.lineplot(x=gremployment_df.get_group(country).year, y=gremployment_df.get_group(country).value/np.mean(gremployment_df.get_group(country).value), label=country)
    
plt.ylabel('employment')
plt.title('Employment in agriculture');

We see an overall downtrend in the agricultural employments.Let's compare the mean employment of the countries:

In [ ]:
plt.figure(figsize=(20,5))
for country in countries:
    plt.bar(x=country, height=np.mean(gremployment_df.get_group(country).value), label = country)
plt.legend()
plt.title('Mean Employment in agriculture')

In order to have a meaningfull comparison the above values should be devided by the total population, but in an absolute comparison we can see employment in agricultural sectors is rather big in China and India.

Compared to the bar chart of agricultural lands above, we notice United States for example that has a pretty vast agricultural land area but has very little employments in agriculture in comparison. 

In [ ]:
remployment_df.to_pickle('employment.pkl')

### Non fatal occupational injuries per 100'000 workers by economic activity

We found it hard to find reliable data for the 'quality of life' which could be subjective and not necessarily related to agriculture. Thus, we found the below dataset which describes the amount of non fatal injuries per economic activity, which we found interessting as a way to asses the social quality and safety of agriculture.

https://www.ilo.org/shinyapps/bulkexplorer32/?lang=en&segment=indicator&id=INJ_NFTL_ECO_RT_A

In [ ]:
nonFatalInjuries_df = pd.read_csv('data/non_fatal_injuries.csv')

#look at some useless columns before droping them
print(nonFatalInjuries_df.obs_status.unique())
print(nonFatalInjuries_df.note_classif.unique().tolist())
nonFatalInjuries_df = nonFatalInjuries_df.drop(columns=['obs_status', 'obs_status.label', 'note_classif', 'note_classif.label', 'note_source', 'note_source.label', 'indicator.label', 'note_indicator', 'source', 'source.label'])   # drop useless columns
nonFatalInjuries_df.head(3)

In [ ]:
# rename columns for consistency
nonFatalInjuries_df.rename(columns={'ref_area.label': 'country_or_area', 'time': 'year', 'obs_value': 'value', 'classif1.label': 'activity'}, inplace= True)

# Check our list
print(check_country(countries, nonFatalInjuries_df['country_or_area']))

In [ ]:
print(np.sort(nonFatalInjuries_df['country_or_area'].unique()).tolist())

In [ ]:
# Problem: Canada is REALLY missing...

# Correct for united states
nonFatalInjuries_df.replace(to_replace='United States', value='United States of America', inplace= True)

# Only keep activities related to agriculture
rnonFatalInjuries_df = nonFatalInjuries_df[nonFatalInjuries_df['activity'].str.contains('Agriculture')]

# There are several regions (cities) of China present in the data set but no China as a whole.
# As our value of interest is a rate, we can take the mean of the latter rates for China as a whole
china_injuries = rnonFatalInjuries_df[nonFatalInjuries_df['country_or_area'].str.contains('China')].reset_index()
print(china_injuries['country_or_area'].unique())    # different chinese cities present

In [ ]:
china_injuries.head()

There is only one entry for Macau and its value is 0. So we drop it and represent China by Taiwan.

In [ ]:
china_injuries.drop(index= 0, axis= 0, inplace= True)
china_injuries.replace('Taiwan, China', 'China', inplace= True)

# append with China data
rnonFatalInjuries_df = rnonFatalInjuries_df.append(china_injuries, sort=False, ignore_index= True)

# reduce to our selected countries
rnonFatalInjuries_df = rnonFatalInjuries_df.loc[rnonFatalInjuries_df['country_or_area'].isin(countries)]

# recheck countries
check_country(countries, rnonFatalInjuries_df['country_or_area'])

We now see that also Japan is missing but that is because Japan hasn't Agriculture in its activity column.

In [ ]:
# simple observations
# groupe by country
grnonFatalInjuries_df = rnonFatalInjuries_df.groupby('country_or_area')

plt.figure(figsize=(15,15))
for country in rnonFatalInjuries_df['country_or_area'].unique():
    # plot and normalize by mean
    sns.lineplot(x=grnonFatalInjuries_df.get_group(country).year, y=grnonFatalInjuries_df.get_group(country).value, label=country)
    
plt.ylabel('injuries rate')
plt.title('Non fatal injuries in agriculture per 100000 workers');

We notice that the data for some countries (China, Russia, India, US, Germany) doesn't cover the entire year range. We also observe unusually high values for Switzerland, which makes us question the sanity of the data. This data in its current form, probably might not be usable for further analysis...

In [ ]:
rnonFatalInjuries_df.to_pickle('nonFatalInjuries.pkl')

# **Question 3 - Analysis**

Is it possible to rank some patterns of land usage with social and environmental factors such as employment, life satisfaction (if data are presents), income inequalities (if data are presents), emissions and finally soil quality and sustainability?

As we saw in the data cleaning nootebook, it is messy and difficult to do analysis for more than 5 countries. Therefore, we reduce our subset to the five following countires: China, India, USA, UK, Switzerland

We chose these countries because we were interested in them and thought that we might find interesting results as they are somehow special countries:

- China has had a massive growth and is now world leading.
- India is a big country with a very traditional soul and also huge growth in the recent years.
- USA has always been a world power however it has not been growing as fast as the above countires in the recent years.
- UK has been a leader in Europe and has had several interesting events in the recent years such as Brexit.
- Switzerland has always been rather stable and it's the country we live in :)

In [ ]:
countries = ['China', 'India', 'United States of America', 'United Kingdom', 'Switzerland']

In [ ]:
# read cleaned data:
dataLands         = pd.read_pickle('data/dataLand.pkl')
dataSoil          = pd.read_pickle('data/dataSoil.pkl')
employment        = pd.read_pickle('data/employment.pkl')
lifeExp           = pd.read_pickle('data/lifeExp.pkl')
nonFatatlInjuries = pd.read_pickle('data/nonFatalInjuries.pkl')
addedValue        = pd.read_pickle('data/valueAdded.pkl')

In [ ]:
# reduce the data to selected countries
dataLands  = dataLands.loc[dataLands['country_or_area'].isin(countries)].reset_index(drop=True)
dataSoil   = dataSoil.loc[dataSoil['Country'].isin(countries)].reset_index(drop=True)
employment = employment.loc[employment['country_or_area'].isin(countries)].reset_index(drop=True)
lifeExp    = lifeExp.loc[lifeExp['country_or_area'].isin(countries)].reset_index(drop=True)
injuries   = nonFatatlInjuries.loc[nonFatatlInjuries['country_or_area'].isin(countries)].reset_index(drop=True)
addedValue = addedValue.loc[addedValue['country_or_area'].isin(countries)].reset_index(drop=True)

## Population Data

We also acquired some new data on world population via the link below. We selected our list of countries directly on the website as it's simpler.

https://databank.worldbank.org/reports.aspx?source=2&series=SP.POP.TOTL&country=#

In [ ]:
population = pd.read_csv('data/population.csv').dropna()

In [ ]:
population.drop(columns= ['Country Code', 'Series Code', 'Series Name'], inplace=True)
population

In [ ]:
# the columns above are ugly: we create a dictionary to replace them with values afterwards
col_years = dict((col,year) for col,year in zip(population.columns[1:], np.arange(1960,2019)))

In [ ]:
population = population.melt(id_vars=['Country Name'], var_name= 'year', value_name= 'value')
population.replace(col_years, inplace=True)

# rename USA
population.replace({'United States':'United States of America'}, inplace=True)
population.head(6)

In [ ]:
grPopulation = population.groupby('Country Name')

plt.figure(figsize=(10,10))
for country in countries:
    
    sns.lineplot(x=grPopulation.get_group(country).year, y=grPopulation.get_group(country).value, label=country)
    
plt.ylabel('Population')
plt.title('Relative evolution of popularion');

We can see that the population of India and China grows really fast compared to USA. The population of UK and Switzerland are pretty satble.

In [ ]:
# groupe by country
grdataLands = dataLands.groupby('country_or_area')

plt.figure(figsize=(10,10))
for country in countries:
    # result list
    norm_land = []
    # we normalize the total land area by population each year
    for year in grdataLands.get_group(country).year.values:
        
        #compute Land / Population ratio for each year
        ratio = grdataLands.get_group(country)[grdataLands.get_group(country).year == year].value.values/ \
                grPopulation.get_group(country)[grPopulation.get_group(country).year == year].value.values
        
        norm_land.append([year, ratio[0]])
        
    sns.lineplot(x=[row[0] for row in norm_land], y=[row[1] for row in norm_land], label=country, estimator=None)
    
plt.ylabel('land area / population')
plt.title('Relative evolution of agricultural land areas');

We see some strange kinks in Switzerland; Let's investigate

In [ ]:
# Switzerland agricultural land and population
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
sns.lineplot(x=grPopulation.get_group('Switzerland').year, y=grPopulation.get_group('Switzerland').value, label='Switzerland', ax=ax[0])
ax[0].set_title('Population')
sns.lineplot(x=grdataLands.get_group('Switzerland').year, y=grdataLands.get_group('Switzerland').value, label='Switzerland', ax=ax[1])
ax[1].set_title('Agricultural Lands');

The problem comes from the Land data; According to Encyclopedia.com :"Swiss agricultural policy is highly regulated, with fixed prices and quota restrictions maintained on several products. Domestic production is encouraged by the imposition of protective customs and duties on imported goods, and by restrictions on imports. The Federal Council has the authority to fix prices of bread grains, flour, milk, and other foodstuffs. Production costs in Switzerland, as well as international exchange rates favorable to the Swiss franc, make competition with foreign products difficult. This highly protectionist system has led to excess production and mounting costs associated with the management of surpluses. The Uruguay Round and subsequent Swiss implementation of its provisions in July 1995 (along with rising costs in the agricultural sector) has forced the government to begin reforming its agricultural support system."

This could be a potential explanation for this behavior, however the kinks in the graph don't really matter for the overall analysis.

We will now investage agricultural employment for each country, comparing male and female employment. As China has only reported the total employments (no male/female separation) it doesn't figure in the first subplots.

In [ ]:
# groupe by country
grEmployment = employment.groupby(['country_or_area','sex.label'])

plt.figure(figsize=(10,10))
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,5))

for i,country in enumerate(['India', 'United States of America', 'United Kingdom', 'Switzerland']):
    
    sns.lineplot(x=grEmployment.get_group((country, 'male')).year, y=grEmployment.get_group((country, 'male')).value, label='Male', ax=ax[i])
    sns.lineplot(x=grEmployment.get_group((country, 'female')).year, y=grEmployment.get_group((country, 'female')).value, label='Female', ax=ax[i])
    ax[i].set_title('Agricultural employment in '+country)

We observe several interesting facts:
- The gap between male and female employment in agriculture decreases over time for USA, UK and Switzerland but stays rather stable for India.
- Comparing males and females employment, we notice that female agricultural employment stays rather stable over time but the male employment decreases. An explanation for this observation could be the relationship of farmers of different sexes with "technology" and automatization. In fact, the major reason of the decrease in agricultural employment could be industrialization and the automatization of agriculture using technology and machines:

    - The male's labor in agriculture back in time has been gradually replaced by machines.
    - Agricultural industrialization had lower impact on females labor as they probably were not the ones harvesting the plants.
    
    
- We also notice a strange downward kink in USA's chart right before year 2000. This could be due to a big technological leap in agriculture but it is most likely due to the dot-com bubble as many people during that year converted to 'entrepreneurs' creating random websites and easiliy obtain fundings and revenue.

In [ ]:
# total aggricultural employment
plt.figure(figsize=(10,10))
for country in countries:
    # store normalized employment in a list
    norm_employment = []
    
    # we normalize the total employment by population each year
    for year in grEmployment.get_group((country, 'total')).year.values:
        
        if year >= 1960:   # population starting year
            #compute Land / Population ratio for each year
            ratio = grEmployment.get_group((country, 'total'))[grEmployment.get_group((country, 'total')).year == year].value.values/ \
                    grPopulation.get_group(country)[grPopulation.get_group(country).year == year].value.values

            norm_employment.append([year, ratio[0]])
        
    sns.lineplot(x=[row[0] for row in norm_employment], y=[row[1] for row in norm_employment], label=country, estimator=None)
    
plt.ylabel('employment / population')
plt.title('Relative evolution of agricultural employment');

We observe that China has the highest ratio of relative agricultural employment followed by India, Switzerland, USA and UK.

### Life Expectancy

In [ ]:
grLifeExp = lifeExp.groupby('country_or_area')

plt.figure(figsize=(10,10))
for country in countries:
    sns.lineplot(x=grLifeExp.get_group(country).year, y=grLifeExp.get_group(country).value, label=country)
    
plt.ylabel('Life expectancy')
plt.title('Evolution of life expectancy');

## Economic Added Value

In [ ]:
grAddedValue = addedValue.groupby('country_or_area')

plt.figure(figsize=(10,10))
for country in countries:
    # store normalized employment in a list
    norm_addedValue = []
    
    # we normalize the total employment by population each year
    for year in grAddedValue.get_group(country).year.values:
        
        if year >= 1960:   # population starting year
            #compute Land / Population ratio for each year
            ratio = grAddedValue.get_group(country)[grAddedValue.get_group(country).year == year].value.values/ \
                    grPopulation.get_group(country)[grPopulation.get_group(country).year == year].value.values

            norm_addedValue.append([year, ratio[0]])
        
    sns.lineplot(x=[row[0] for row in norm_addedValue], y=[row[1] for row in norm_addedValue], label=country, estimator=None)
    
plt.ylabel('economic added value')
plt.title('Agriculture economic added value');

It is interesting to notice that india has the largest economical added value via agriculture, however it has the smallest agricultural land to population ratio.